<a href="https://colab.research.google.com/github/mastgm0817/project_DL/blob/main/mastgm0817/%EC%98%81%ED%99%94%ED%8F%89%EC%A0%90%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!chmod 600 /content/kaggle.json
!mkdir ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d raman77768/movie-classifier

100% 247M/248M [00:12<00:00, 24.0MB/s]
100% 248M/248M [00:12<00:00, 20.8MB/s]


In [ ]:
!unzip movie-classifier.zip

In [ ]:
!pip install scikit-multilearn
!pip install tensorflow
!pip install scikit-learn==0.24.1

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import os
from tqdm import tqdm, tqdm_notebook
from torch.autograd import Variable
from torchsummary import summary

In [7]:
df = pd.read_csv('/content/Multi_Label_dataset/train.csv')
df = df.head(2300)
df = df.dropna()
df

,Id,Genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,N/A,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
0,tt0086425,"['Comedy', 'Drama']",0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,tt0085549,"['Drama', 'Romance', 'Music']",0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,tt0086465,['Comedy'],0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0086567,"['Sci-Fi', 'Thriller']",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,tt0086034,"['Action', 'Adventure', 'Thriller']",1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,tt0116130,['Comedy'],0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2296,tt0116225,"['Action', 'Adventure', 'Sci-Fi']",1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2297,tt0116329,"['Adventure', 'Drama', 'Family']",0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2298,tt0116778,"['Comedy', 'Sport']",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [10]:
from torchvision import transforms
from PIL import Image
# 이미지 불러오기
width = 400
height = 300
transform = transforms.Compose([
    transforms.Resize((width, height)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # RGB 값의 평균과 표준편차
])
X = []
for i in tqdm(range(df.shape[0])):
    path = '/content/Multi_Label_dataset/Images/' + df['Id'][i] + '.jpg'
    img = Image.open(path)
    img = transform(img)
    img = img.numpy()
#     img = np.transpose(img, (1,2,0))
    X.append(img)

100%|██████████| 2300/2300 [00:16<00:00, 141.74it/s]


In [ ]:
from sklearn.model_selection import train_test_split

X = np.array(X)

y = df.drop(['Id','Genre'],axis=1)
y = y.to_numpy()

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)
# X_train = np.transpose(X_train, (0,2,3,1))


In [9]:
X_train.shape, y_train.shape

((2070, 400, 300, 3), (2070, 25))

In [6]:
def atleastOneMatch(y_true, y_pred):
    total_count=len(y_true)
    total_match=0
    for i in range(0, len(y_true)):
        if(np.any(y_true[i][y_pred[i]==1])):
            total_match+=1
    return total_match/total_count
            
def allMatch(y_true, y_pred):
    total_count=len(y_true)
    total_match=0
    for i in range(y_true.shape[0]):
        if(np.all(y_true[i]==y_pred[i])):
            total_match+=1
    return total_match/total_count

def accuracy_score(y_true, y_pred):
    y_pred = np.concatenate(tuple(y_pred))
    y_true = np.concatenate(tuple([[t for t in y] for y in y_true])).reshape(y_pred.shape)
    return (y_true == y_pred).sum() / float(len(y_true))

def split_list(A, n):
    f = [A[i:i+n] for i in range(0, len(A), n)]
    return f

In [7]:
n_epochs = 5
learning_rate = 1e-03
batch_size = 64
# Print all of the hyperparameters of the training iteration:
print("===== HYPERPARAMETERS =====")
print("epochs=", n_epochs)
print("learning_rate=", learning_rate)
print("batch size=", batch_size)
print("=" * 30)

===== HYPERPARAMETERS =====
epochs= 5
learning_rate= 0.001
batch size= 64


In [7]:
class SimpleCNN(torch.nn.Module):

    # Our batch shape for input x is (1, number_of_unique_kos, genome_size)

    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(5,5)).to(device)
        nn.init.xavier_uniform_(self.conv1.weight)
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2)).to(device)
        self.drop1 = nn.Dropout2d(p=0.25).to(device)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5,5)).to(device)
        nn.init.xavier_uniform_(self.conv2.weight)
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2)).to(device)
        self.drop2 = nn.Dropout2d(p=0.25).to(device)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5,5)).to(device)
        nn.init.xavier_uniform_(self.conv3.weight)
        self.pool3 = nn.MaxPool2d(kernel_size=(2,2)).to(device)
        self.drop3 = nn.Dropout2d(p=0.25).to(device)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,5)).to(device)
        nn.init.xavier_uniform_(self.conv4.weight)
        self.pool4 = nn.MaxPool2d(kernel_size=(2,2)).to(device)
        self.drop4 = nn.Dropout2d(p=0.25).to(device)

        # self.fc1 = torch.nn.Linear(16384, 128).to(device)
        self.fc1 = torch.nn.Linear(64 * 15 * 21, 128).to(device)
        nn.init.xavier_uniform_(self.fc1.weight)
        self.drop5 = nn.Dropout(p=0.5).to(device)

        self.fc2 = torch.nn.Linear(128, 64).to(device)
        nn.init.xavier_uniform_(self.fc2.weight)
        self.drop6 = nn.Dropout(p=0.5).to(device)

        self.fc3 = torch.nn.Linear(64, 25).to(device)
        nn.init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.drop1(x)

        x = F.relu(self.conv2(x))  
        x = self.pool2(x)  
        x = self.drop2(x)             

        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = self.drop3(x)

        x = F.relu(self.conv4(x))
        x = self.pool4(x)
        x = self.drop4(x) 

        # x = x.reshape(-1, 16384)
        x = x.reshape(-1, 64 * 15 * 21)
        x = self.fc1(x)
        x = self.drop5(x) 
        x = self.fc2(x)
        x = self.drop6(x) 
        x = self.fc3(x)

        return torch.sigmoid(x)

In [9]:
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

In [8]:
net = SimpleCNN()
# summary(net, (3,400,300))

NameError: ignored

In [16]:
running_loss = 0.0
net = SimpleCNN()

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
loss = torch.nn.BCELoss().to(device)

batches = split_list(X_train, batch_size)
batches_ind = split_list(range(len(X_train)), batch_size)
n_batches = len(batches)

N = len(X_train)
print(X_train.shape)
training_start_time = time.time()
for epoch in tqdm(range(n_epochs)):
    
    running_loss = 0.0
    print_every = 15 # N // 50
    start_time = time.time()
    total_train_loss = 0
    total_output = torch.zeros(len(X_train),25)
    
    for ind in range(n_batches):
        
        batch_elements = [el for el in batches[ind]]
        batch_ind = [el for el in batches_ind[ind]]
        

#         input_data = torch.tensor(load_data(batch_elements))
        input_data = torch.tensor(batches[ind])
        
        if len(input_data) != batch_size:
            batch_size = len(input_data)
        
        input_data = torch.tensor(input_data, dtype=torch.float32).to(device)

        labels = torch.from_numpy(y[batch_ind]).to(device)
        optimizer.zero_grad()

        # Forward pass, backward pass, optimize
        output = net(input_data)
        total_output[torch.tensor(batch_ind),:] = output.cpu().detach()
        loss_size = loss(output, labels.float())
        loss_size.backward()
        optimizer.step()   
        
        #Print statistics
        running_loss += loss_size.data.item()
        total_train_loss += loss_size.item()
        
        if ind % print_every == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.4f} took: {:.4f}s".format(
                epoch + 1, int(100 * (ind + 1) / n_batches), running_loss / print_every, time.time() - start_time))
            # Reset running loss and time
            running_loss = 0.0
            start_time = time.time()
    train_acc = accuracy_score(y_train,(total_output>0.5).data.numpy())
    print("\n" + "Train Accuracy: {:.4f}".format(train_acc))     

(2070, 400, 300, 3)


<ipython-input-16-bf04145bf6a1>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(n_epochs)):


  0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-16-bf04145bf6a1>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(input_data, dtype=torch.float32).reshape(-1, 3, width, height).to(device)


Epoch 1, 3% 	 train_loss: 0.0583 took: 8.0838s
Epoch 1, 48% 	 train_loss: 0.4832 took: 139.0570s
Epoch 1, 93% 	 train_loss: 0.3610 took: 129.4708s

Train Accuracy: 0.8236
Epoch 2, 3% 	 train_loss: 0.0219 took: 8.6622s
Epoch 2, 48% 	 train_loss: 0.3161 took: 131.5692s
Epoch 2, 93% 	 train_loss: 0.2965 took: 132.1700s

Train Accuracy: 0.8910
Epoch 3, 3% 	 train_loss: 0.0189 took: 9.2718s
Epoch 3, 48% 	 train_loss: 0.2900 took: 132.7975s
Epoch 3, 93% 	 train_loss: 0.2788 took: 132.1944s

Train Accuracy: 0.8984
Epoch 4, 3% 	 train_loss: 0.0179 took: 7.6113s
Epoch 4, 48% 	 train_loss: 0.2795 took: 131.8947s
Epoch 4, 93% 	 train_loss: 0.2712 took: 132.5688s

Train Accuracy: 0.9021
Epoch 5, 3% 	 train_loss: 0.0179 took: 8.8424s
Epoch 5, 48% 	 train_loss: 0.2709 took: 132.9568s
Epoch 5, 93% 	 train_loss: 0.2633 took: 131.7125s

Train Accuracy: 0.9028


In [ ]:
test_data = torch.tensor(X_test)
test_data = Variable(test_data).to(device)
y_test_pred = net(test_data)
prediction=(y_test_pred.cpu().detach()>0.3).data.numpy()
test_acc = accuracy_score(y_test,prediction)
print("\n" + "Test Accuracy: {:.4f}".format(test_acc)) 
print(f"Aleast one correct: {atleastOneMatch(y_test, prediction)*100} %")

In [ ]:
def predict(cnn, location):

#     img = image.load_img(location,target_size=(400,300,3))
#     img = image.img_to_array(img)
#     img = img/255

#     img =  Variable(torch.from_numpy(img)).transpose(0,2).transpose(1,2).to(device)
#     img = img.view(1,3,400,-1)

    transform = transforms.Compose([
                    transforms.Resize((300, 400)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                ])

    img = Image.open(location)
    img = transform(img).unsqueeze(0).to(device)
    proba = cnn(img)

    classes = np.array(df.columns[2:])
    proba = proba.cpu().detach().numpy()
    top_3 = np.argsort(proba[0])[:-4:-1]
    for i in range(3):
        print("{}".format(classes[top_3[i]])+" ({:.3})".format(proba[0][top_3[i]]))
        
        
    

In [ ]:
img = Image.open('/content/new_img/타이타닉.jpeg')
img = img.resize((400, 300))
plt.imshow(img)
predict(net,'/content/new_img/타이타닉.jpeg')